## Non Compliance Experiment=1
Test top norms for different w_nc

In [91]:
from IPython.utils import io
from tqdm.notebook import tqdm
from pathlib import Path

In [89]:
def write_log(output,path):
    with open(Path(path), 'w') as f:
        f.write(output)

In [95]:
%%time
with io.capture_output() as exp_nc_0_log:
    !python ../scripts/nc_experiments.py -exp exp_nc0 -w_nc 0.7 -n_threads 3
write_log(exp_nc_0_log.stdout,"../data_nc/exp_nc0/exp_nc_0_run.log")  

CPU times: user 8.72 s, sys: 1.97 s, total: 10.7 s
Wall time: 38min 53s


In [ ]:
!dvc add ../data_nc/exp_nc0/
!dvc push

In [34]:
!git add --all
!git commit -m "Results gen for exp_nc_0"

In [17]:
w_nc_list=[0.0,0.05,0.1,0.2,0.3,0.4]

In [ ]:
for w_nc in tqdm(w_nc_list,"Loop Over diff w_nc"):
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    with io.capture_output() as exp_nc_0_log:
        !python ../scripts/nc_experiments.py -exp $exp_dir_name -w_nc 0.7 -n_threads 6
    write_log(exp_nc_0_log.stdout,"../data_nc/exp_nc0/exp_nc_0_run.log")  